In [ ]:
!pip install spacy

In [ ]:
import spacy
import nltk
from nltk import sent_tokenize


In [ ]:
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
def load_model():
  nlp = spacy.load("en_core_web_trf")
  return nlp

In [ ]:
!pip install spacy-curated-transformers

In [ ]:
!pip install spacy-transformers


In [ ]:
import spacy_transformers

nlp_model=load_model()

/usr/local/lib/python3.11/dist-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map_location=dev

In [ ]:
from glob import glob
files = glob("/content/data/*.ass")

In [ ]:
files[:2]

['/content/data/Naruto Season 2 - 47.ass',
 '/content/data/Naruto Season 1 - 05.ass']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import pathlib

In [ ]:
## A function to load the data from the subtitles files and combine them into one dictionary {episode:script of the episode} at that fromat

def load_subs(data_path):
  subs=glob(data_path+'/*.ass')
  scripts = []
  episodes=[]

  for path in subs:

    with open(path,'r') as file:
      lines = file.readlines()
      lines = lines[27:] # As in every file the first 27 line is not needed
      lines = [",".join(line.split(',')[9:]) for line in lines] # cutting everything before the 9th comma
    lines = [line.replace('\\N'," ") for line in lines]
    script = " ".join(lines) # Combining the scripts of the episode into one script
    scripts.append(script)
    episode = int(path.split('-')[-1].split('.')[0].strip())
    episodes.append(episode)

  df = pd.DataFrame.from_dict({'episode':episodes,'script':scripts})
  return df


In [ ]:
sub_path = "/content/data"
df = load_subs(sub_path)

In [ ]:
df.head()

,episode,script
0,47,"Press down hard on the gas\n That’s right, the..."
1,5,"C'mon!\n Running like a fugitive,\n Being chas..."
2,92,We are Fighting Dreamers aiming high\n Fightin...
3,25,"C'mon!\n Running like a fugitive,\n Being chas..."
4,104,Connecting old words that have been used up\n ...


In [ ]:
df=df.sort_values(by='episode')

In [ ]:
df.head()

,episode,script
137,1,"A long time ago, a powerful demon fox appeared..."
16,2,"C'mon!\n Running like a fugitive,\n Being chas..."
80,3,"C'mon!\n Running like a fugitive,\n Being chas..."
210,4,"C'mon!\n Running like a fugitive,\n Being chas..."
1,5,"C'mon!\n Running like a fugitive,\n Being chas..."


In [ ]:
sample_script=df.iloc[0]['script']

In [ ]:
sample_script

'A long time ago, a powerful demon fox appeared with nine tails.\n With its powerful tails,\n it could smash mountains and create tidal waves.\n A band of Ninjas rose to defend their village from attack.\n We have to wait until the Fourth Hokage gets here!\n We can\'t let it get any closer to our village!\n One great Ninja was able to imprison the monster,\n but died in the process.\n This Ninja was known as… the Fourth Hokage.\n Naruto!\n Why did you do such a thing?!\n You\'re really gonna get it this time!\n I don\'t care!\n You know your problem?\n You can\'t do the things I do!\n Only I can do this!\n I\'m better than all of you! Believe it!\n There\'s a problem, sir!\n Lord Hokage!\n What is it?\n Did that Naruto do something again?\n Yes. He climbed onto the Mountainside Images…\n And he vandalized and graffitied all over them!\n Wait!\n Ha ha…\n Why should I?\n Hey, Naruto!\n How did you suddenly get here, lruka Sensei?\n The question is what are you doing here when you should 

In [ ]:
nltk.download('punkt_tab')
sentences = sent_tokenize(sample_script)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
sentences=sentences[50:90]

In [ ]:
sentence=".".join(sentences)

In [ ]:
doc=nlp_model(sentence)

/usr/local/lib/python3.11/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


In [ ]:
doc.ents

(Naruto,
 tonight,
 Naruto Uzumaki!.Naruto,
 Ninja,
 Fourth,
 nine,
 Sensei,
 Leaf,
 Ninja Academy,
 Tomorrow,
 One more bowl,
 three,
 Naruto,
 one)

In [ ]:
# This is a function to extract only the entities of the characters and get the first name only without any duplicates
def get_ners(script):
  script_sentences = sent_tokenize(script)

  ner_output = []

  for sentence in script_sentences :
    doc = nlp_model(sentence)
    ners = set() # To avoid any duplicates

    for ent in doc.ents:
      if ent.label_ == "PERSON":
        full_name = ent.text
        first_name = ent.text.split(" ")[0]
        first_name = first_name.strip()
        ners.add(first_name)
    ner_output.append(ners)
  return ner_output

In [ ]:
# Running it on 10 samples

df['ners'] = df['script'].head(10).apply(get_ners)

/usr/local/lib/python3.11/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


In [ ]:
!pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.0 MB/s eta 0:00:00


In [ ]:
import networkx as nx
from pyvis.network import Network

In [ ]:
def generate_character_network(df):
# A function to create the network
    windows=10
    entity_relationship = []

    for row in df['ners']: # We will go row by row on each sentence
        previous_entities_in_window = []

        for sentence in row:
            previous_entities_in_window.append(list(sentence))
            previous_entities_in_window = previous_entities_in_window[-windows:] # Taking the first 10 sentence
            #  so it is like that here [[char1],[char2],...]
            # Flatten 2D List into 1D List
            previous_entities_flattened = sum(previous_entities_in_window, [])
            # It has become like that [char1,char2]
            for entity in sentence:
                for entity_in_window in previous_entities_flattened:
                    if entity != entity_in_window:
                        entity_relationship.append(sorted([entity, entity_in_window])) # Making pairs of entities --> [char1,char2] char1 was talking to char2


    relationship_df = pd.DataFrame({'value': entity_relationship})
    relationship_df['source'] = relationship_df['value'].apply(lambda x: x[0])
    relationship_df['target'] = relationship_df['value'].apply(lambda x: x[1])
    relationship_df = relationship_df.groupby(['source', 'target']).count().reset_index()
    relationship_df = relationship_df.sort_values('value', ascending=False)

    return relationship_df

In [ ]:
sample = df.head(10)

In [ ]:
relationship_df = generate_character_network(sample)

In [ ]:
relationship_df

,source,target,value
125,Naruto,Sasuke,117
152,Sakura,Sasuke,65
67,Iruka,Naruto,43
124,Naruto,Sakura,41
118,Mizuki,Naruto,28
...,...,...,...
98,Kakashi,Sharingan,1
91,Jonin,Zabuza,1
87,Jonin,Manji,1
75,Jerk,Sakura,1


In [ ]:
relationship_df= relationship_df.sort_values(by='value',ascending=False)
relationship_df = relationship_df.head(200)

In [ ]:
G=nx.from_pandas_edgelist(relationship_df,source='source',target='target',edge_attr='value',create_using=nx.Graph()
)

net = Network(notebook=True,height="700px",bgcolor='#222222',font_color="white",cdn_resources="remote")
node_degree = dict(G.degree)
nx.set_node_attributes(G,node_degree,'size')
net.from_nx(G)
net.show("character_network.html")

character_network.html
